<a href="https://colab.research.google.com/github/harshaadeshmukh/Shabancha-shunya/blob/main/Marathi_Sign_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
%cd /content/drive/MyDrive
dataset_path = 'data'

/content/drive/MyDrive


In [ ]:
# Create ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,  # Avoid flips as signs may be orientation-specific
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

In [ ]:
# Load and prepare data generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # EfficientNetB0 input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 40086 images belonging to 43 classes.
Found 10013 images belonging to 43 classes.


In [ ]:
# Save class indices for later use in inference
with open('class_indices.json', 'w', encoding='utf-8') as f:
    json.dump(train_generator.class_indices, f)

In [ ]:
# Load the pre-trained EfficientNetB0 model (without top layers)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model for initial training

# Add custom layers for classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Add dropout for regularization
predictions = layers.Dense(43, activation='softmax')(x)  # 43 classes for Marathi letters

# Create the final model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define callbacks for training
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

backup_callback = tf.keras.callbacks.BackupAndRestore(
    backup_dir='backup_dir',
    save_freq=100
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=1,
    callbacks=[checkpoint, early_stopping, backup_callback]
)

# Save the final model
model.save('marathi_sign_language_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1252/1252 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9669 - loss: 0.1072
Epoch 1: val_accuracy improved from -inf to 0.96675, saving model to best_model.h5


1252/1252 ━━━━━━━━━━━━━━━━━━━━ 12990s 10s/step - accuracy: 0.9669 - loss: 0.1072 - val_accuracy: 0.9667 - val_loss: 0.1096
Restoring model weights from the end of the best epoch: 1.


In [ ]:
# Plot training history (optional, for visualization)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Inference function (save this in a separate script or continue here)
def predict_sign(image_path, model_path='marathi_sign_language_model.h5'):
    # Load the saved class indices
    with open('class_indices.json', 'r') as f:
        class_indices = json.load(f)
    inv_map = {v: k for k, v in class_indices.items()}  # Invert for mapping indices to labels

    # Load the trained model
    model = tf.keras.models.load_model(model_path)

    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    predicted_letter = inv_map[predicted_class]

    return predicted_letter, predictions[0]  # Return letter and probabilities for confidence

# Example usage of inference
if __name__ == "__main__":
    test_image_path = 'data/अ/0.jpg'  # Replace with your test image path
    predicted_letter, probabilities = predict_sign(test_image_path)
    print(f'The predicted letter is: {predicted_letter}')
    print(f'Confidence (probability): {np.max(probabilities):.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The predicted letter is: अ
Confidence (probability): 0.9987
